## Базовый анализ и расширение данных

In [1]:
#Импорт модулей
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

In [ ]:
#Чтение файла
taxi_data = pd.read_csv(r"d:\IDE_github\mathml\data\train.csv")
print('Train data shape: {}'.format(taxi_data.shape))
taxi_data.head()

Train data shape: (1458644, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


**Данные о клиенте и таксопарке:**
* id - уникальный идентификатор поездки
* vendor_id - уникальный идентификатор поставщика (таксопарка), связанного с записью поездки

**Временные характеристики:**
* pickup_datetime - дата и время, когда был включен счетчик поездки
* dropoff_datetime - дата и время, когда счетчик был отключен

**Географическая информация:**
* pickup_longitude -  долгота, на которой был включен счетчик
* pickup_latitude - широта, на которой был включен счетчик
* dropoff_longitude - долгота, на которой счетчик был отключен
* dropoff_latitude - широта, на которой счетчик был отключен

**Прочие признаки:**
* passenger_count - количество пассажиров в транспортном средстве (введенное водителем значение)
* store_and_fwd_flag - флаг, который указывает, сохранилась ли запись о поездке в памяти транспортного средства перед отправкой поставщику. Y - хранить и пересылать, N - не хранить и не пересылать поездку.

**Целевой признак:**
* trip_duration - продолжительность поездки в секундах

### Временные рамки

In [4]:
taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
min_data = min(taxi_data['pickup_datetime'].values)
max_data = max(taxi_data['pickup_datetime'].values)
print(f'Временные рамки:\nрання дата {min_data}\nпоздняя дата {max_data}')

Временные рамки:
рання дата 2016-01-01T00:00:17.000000000
поздняя дата 2016-06-30T23:59:39.000000000


### Пропущенные значения

In [5]:
empty_values_sum = taxi_data.isna().sum()
#Столбцы с пропусками
columns_with_missing = empty_values_sum[empty_values_sum > 0]
if columns_with_missing.empty:
    print('Пропусков в данных нет')
else:
    print('Столбцы с пропусками:')
    for column, count in columns_with_missing.items():
        print(f'  {column}: {count} пропусков')

Пропусков в данных нет


### Статистические характеристики

In [6]:
#Количество уникальных таксопарков
unique_vendors = taxi_data['vendor_id'].nunique()
print(f"Уникальных таксопарков: {unique_vendors}")
#Максимальное количество пассажиров
max_passengers = taxi_data['passenger_count'].max()
print(f"Максимальное количество пассажиров: {max_passengers}")
#Средняя и медианная длительность поездки (округлено до целого)
mean_duration = round(taxi_data['trip_duration'].mean())
median_duration = round(taxi_data['trip_duration'].median())
print(f"Средняя длительность: {mean_duration} секунд")
print(f"Медианная длительность: {median_duration} секунд")
#Минимальное и максимальное время поездки
min_duration = taxi_data['trip_duration'].min()
max_duration = taxi_data['trip_duration'].max()
print(f"Минимальное время поездки: {min_duration} секунд")
print(f"Максимальное время поездки: {max_duration} секунд")

Уникальных таксопарков: 2
Максимальное количество пассажиров: 9
Средняя длительность: 959 секунд
Медианная длительность: 662 секунд
Минимальное время поездки: 1 секунд
Максимальное время поездки: 3526282 секунд


Судя по последним характеристикам, в данных есть выбросы

### Даты поездок

In [7]:
def add_datetime_features(df):
    """
    Добавляет в DataFrame признаки из datetime.
    Parameters:
    df (pd.DataFrame): Исходный DataFrame с колонкой 'pickup_datetime'
    Returns:
    pd.DataFrame: DataFrame с добавленными колонками
    """
    #Создаем копию DataFrame чтобы избежать предупреждений
    df = df.copy()
    #Добавляем новые признаки
    df['pickup_date'] = df['pickup_datetime'].dt.date
    df['pickup_hour'] = df['pickup_datetime'].dt.hour
    df['pickup_day_of_week'] = df['pickup_datetime'].dt.dayofweek  # 0-понедельник, 6-воскресенье
    return df

#Применяем функцию к данным
taxi_data = add_datetime_features(taxi_data)

#Поездки в субботу (день недели = 5, т.к. понедельник=0, воскресенье=6)
saturday_trips = taxi_data[taxi_data['pickup_day_of_week'] == 5].shape[0]
print(f"Поездок в субботу: {saturday_trips}")
#Среднее количество поездок в день
trips_per_day = taxi_data.groupby('pickup_date').size()  #Количество поездок по дням
avg_trips_per_day = round(trips_per_day.mean())
print(f"Среднее количество поездок в день: {avg_trips_per_day}")

Поездок в субботу: 220868
Среднее количество поездок в день: 8015


### Создание признаков на датасете о праздниках

In [9]:
holiday_data = pd.read_csv(r'd:\IDE_github\mathml\data\holiday_data.csv', sep=';')
def add_holiday_features(taxi_df, holiday_df):
    """
    Добавляет признак праздничного дня в DataFrame с поездками.
    
    Parameters:
    taxi_df (pd.DataFrame): DataFrame с поездками и колонкой 'pickup_date'
    holiday_df (pd.DataFrame): DataFrame с праздничными днями
    
    Returns:
    pd.DataFrame: DataFrame с добавленным столбцом 'pickup_holiday'
    """
    #Создаем копию чтобы избежать предупреждений
    taxi_df = taxi_df.copy()
    #Преобразование в формат .datetime
    holiday_df['date'] = pd.to_datetime(holiday_df['date'])
    #Создаем множество праздничных дат для быстрого поиска
    holiday_dates = set(holiday_df['date'].dt.date)
    #Добавляем бинарный признак праздника
    taxi_df['pickup_holiday'] = taxi_df['pickup_datetime'].dt.date.isin(holiday_dates).astype(int)
    return taxi_df

#Применяем функцию к данным
taxi_data = add_holiday_features(taxi_data, holiday_data)

#Фильтруем только праздничные поездки и вычисляем медиану
holiday_trips = taxi_data[taxi_data['pickup_holiday'] == 1]
median_holiday_duration = round(holiday_trips['trip_duration'].median())
print(f"Медианная длительность поездки в праздничные дни: {median_holiday_duration} секунд")

Медианная длительность поездки в праздничные дни: 585 секунд


### Оптимальный путь из точек

In [12]:
osrm_data = pd.read_csv(r'd:\IDE_github\mathml\data\osrm_data_train.csv')

def add_osrm_features(taxi_df, osrm_df):
    """
    Добавляет OSRM-признаки в DataFrame с поездками.
    
    Parameters:
    taxi_df (pd.DataFrame): DataFrame с поездками
    osrm_df (pd.DataFrame): DataFrame с OSRM данными
    
    Returns:
    pd.DataFrame: DataFrame с добавленными OSRM столбцами
    """
    #Создаем копию чтобы избежать предупреждений
    taxi_df = taxi_df.copy()
    #Объединяем таблицы по id поездки
    taxi_df = taxi_df.merge(
        osrm_df[['id', 'total_distance', 'total_travel_time', 'number_of_steps']],
        on='id',
        how='left'  #left join чтобы сохранить все поездки из taxi_df
    )
    return taxi_df

#Применяем функцию к данным
taxi_data = add_osrm_features(taxi_data, osrm_data)

In [13]:
def get_haversine_distance(lat1, lng1, lat2, lng2):
    #Переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    #Радиус земли в километрах
    EARTH_RADIUS = 6371 
    #Считаем кратчайшее расстояние h по формуле Хаверсина
    lat_delta = lat2 - lat1
    lng_delta = lng2 - lng1
    d = np.sin(lat_delta * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng_delta * 0.5) ** 2
    h = 2 * EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def get_angle_direction(lat1, lng1, lat2, lng2):
    #Переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    #Считаем угол направления движения alpha по формуле угла пеленга
    lng_delta_rad = lng2 - lng1
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    alpha = np.degrees(np.arctan2(y, x))
    return alpha

In [14]:
#Разница между медианной длительностью поездки в данных и из OSRM
median_actual = taxi_data['trip_duration'].median()
median_osrm = taxi_data['total_travel_time'].median()
median_difference = abs(median_actual - median_osrm)
print(f"Разница между медианными длительностями: {median_difference:.0f} секунд")
#Количество пропусков в OSRM столбцах
missing_osrm = taxi_data[['total_distance', 'total_travel_time', 'number_of_steps']].isna().sum()
print("\nПропуски в OSRM столбцах:")
for col, count in missing_osrm.items():
    print(f"   {col}: {count} пропусков")

Разница между медианными длительностями: 372 секунд

Пропуски в OSRM столбцах:
   total_distance: 1 пропусков
   total_travel_time: 1 пропусков
   number_of_steps: 1 пропусков
